In [1]:
import cv2
from ultralytics import YOLO
import datetime

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # width
cap.set(4, 720)   # height


# Video writer setup (recording output)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(f'output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.avi', fourcc, 20.0, (1280, 720))

# Define zones (e.g., 3x3 grid)
rows, cols = 3, 3
zone_width = int(1280 / cols)
zone_height = int(720 / rows)

def draw_zones(frame):
    for i in range(1, cols):
        cv2.line(frame, (i * zone_width, 0), (i * zone_width, 720), (0, 255, 0), 1)
    for j in range(1, rows):
        cv2.line(frame, (0, j * zone_height), (1280, j * zone_height), (0, 255, 0), 1)

def get_zone(x, y):
    col = x // zone_width
    row = y // zone_height
    return int(row * cols + col)  # Zone index

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame, stream=True)

    draw_zones(frame)  # Draw grid zones

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls]

            # Only track people
            if label == "person":
                # Calculate center of the box
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                # Identify zone
                zone = get_zone(cx, cy)

                # Hacker green style box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} Z{zone}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 2)

                # Mark center point
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

    out.write(frame)  # Record frame to video file
    cv2.imshow("YOLOv8 Zone Detection - Hacker Mode", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 130.9ms
Speed: 7.2ms preprocess, 130.9ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 72.9ms
Speed: 3.4ms preprocess, 72.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 66.2ms
Speed: 1.7ms preprocess, 66.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 67.1ms
Speed: 1.5ms preprocess, 67.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 66.5ms
Speed: 1.7ms preprocess, 66.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 61.0ms
Speed: 1.1ms preprocess, 61.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 61.2ms
Speed: 1.1ms preprocess, 61.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 62.9ms
Speed: 1.1ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3,